In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("주제 {topic}에 대해 짧은 설명을 해주세요.")
prompt_template.invoke({"topic":"투자"})

StringPromptValue(text='주제 투자에 대해 짧은 설명을 해주세요.')

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 유능한 금융 조언가입니다."),
    ("user", "주제 {topic}에 대해 금융 관련 조언을 해주세요.")
])
prompt_template.invoke({"topic":"주식"})

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 유능한 금융 조언가입니다."),
    MessagesPlaceholder("msgs"),
])

prompt_template.invoke({"msgs":[HumanMessage(content="안녕하세요!")]})

ChatPromptValue(messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요!', additional_kwargs={}, response_metadata={})])

In [6]:
rompt_template = ChatPromptTemplate.from_messages([
("system", "당신은 유능한 금융 조언가입니다."),
("placeholder", "{msgs}")
])

prompt_template.invoke({"msgs": [HumanMessage(content="안녕하세요!")]})


ChatPromptValue(messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요!', additional_kwargs={}, response_metadata={})])

In [7]:
from langchain_core.prompts import PromptTemplate

example_prompt = PromptTemplate.from_template("질문: {question}\n답변: {answer}")

examples = [
    {
        "question": "주식 투자와 예금 중 어느 것이 더 수익률이 높은가?",
        "answer": """
후속 질문이 필요한가요: 네.
후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
후속 질문: 예금의 평균 이자율은 얼마인가요?
중간 답변: 예금의 평균 이자율은 연 1%입니다.
따라서 최종 답변은: 주식 투자
""",
    } ,
    {
        "question": "부동산과 채권 중 어느 것이 더 안정적인 투자처인가?",
        "answer": """
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권
""",
    },
]

print(example_prompt.invoke(examples[0]).to_string())

질문: 주식 투자와 예금 중 어느 것이 더 수익률이 높은가?
답변: 
후속 질문이 필요한가요: 네.
후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
후속 질문: 예금의 평균 이자율은 얼마인가요?
중간 답변: 예금의 평균 이자율은 연 1%입니다.
따라서 최종 답변은: 주식 투자



In [9]:
from langchain_core.prompts import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="질문: {input}",
    input_variables=["input"],
)

print(prompt.invoke({"input":"부동산 투자의 장점은 무엇인가?"}).to_string())

질문: 주식 투자와 예금 중 어느 것이 더 수익률이 높은가?
답변: 
후속 질문이 필요한가요: 네.
후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
후속 질문: 예금의 평균 이자율은 얼마인가요?
중간 답변: 예금의 평균 이자율은 연 1%입니다.
따라서 최종 답변은: 주식 투자


질문: 부동산과 채권 중 어느 것이 더 안정적인 투자처인가?
답변: 
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권


질문: 부동산 투자의 장점은 무엇인가?


In [10]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    Chroma,
    k=1,
)

question = "부동산 투자의 장점은 무엇인가?"
selected_examples = example_selector.select_examples({"question": question})

print(f"입력과 가장 유사한 예제: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")

입력과 가장 유사한 예제: 부동산 투자의 장점은 무엇인가?


question: 부동산과 채권 중 어느 것이 더 안정적인 투자처인가?
answer: 
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권



In [11]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI

example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="질문: {question}\n답변: {answer}"
)

prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="다음은 금융 관련 질문과 답변의 예시입니다:",
    suffix="질문: {input}\n답변:",
    input_variables=["input"]
)


In [12]:
model = ChatOpenAI(model_name="gpt-4o")

In [13]:
chain = prompt | model

response = chain.invoke({"input": "부동산 투자의 장점은 무엇인가?"})
print(response.content)

부동산 투자의 장점은 다양합니다:

1. **자산 가치 상승 가능성**: 부동산은 시간이 지남에 따라 가치가 상승할 가능성이 있어 장기적으로 자산 증식에 도움이 될 수 있습니다.

2. **안정적인 수익**: 임대를 통해 정기적인 현금 흐름을 창출할 수 있으며, 이는 안정적인 수익원이 될 수 있습니다.

3. **인플레이션 헤지**: 부동산은 일반적으로 인플레이션에 대한 헤지 수단으로 여겨지며, 물가 상승 시 임대료와 자산 가치가 함께 상승할 수 있습니다.

4. **세금 혜택**: 부동산 투자에는 다양한 세금 공제와 혜택이 있어, 투자 수익률을 높이는 데 도움이 됩니다.

5. **실물 자산**: 부동산은 실물 자산으로, 주식이나 채권과 달리 물리적으로 존재하는 자산의 소유권을 제공합니다.

위의 장점들을 고려할 때, 부동산은 장기적인 투자 전략의 일환으로 많은 투자자들 사이에서 인기가 있습니다.


In [14]:
from langsmith import Client

client = Client()
prompt = client.pull_prompt("hardkothari/prompt-maker")
prompt

ChatPromptTemplate(input_variables=['lazy_prompt', 'task'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hardkothari', 'lc_hub_repo': 'prompt-maker', 'lc_hub_commit_hash': 'c5db8eeefa7be4862a9599b759608dd10ee53f53910838f69abb5ab31c257c2d'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert Prompt Writer for Large Language Models.\n\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['lazy_prompt', 'task'], input_types={}, partial_variables={}, template='Your goal is to improve the prompt given below for {task} :\n--------------------\n\nPrompt: {lazy_prompt}\n\n--------------------\n\nHere are several tips on writing great prompts:\n\n-------\n\nStart the prompt by stating that it is an expert in the subject.\n\nPut instructions at the beginning of the prompt and use ### or to separate the instruction and context \n\nBe specific, 